# 2020 -> 2010 Crosswalk

Author: Mo Al Elew

**What notebook does/produces:**

Produces 2022 American Community Survey (ACS) racial demographics data by 2020 block group interpolated to 2010 tracts

**Issue/problem being solved:**

Auto Club sets rate relativities by 2010 tracts and we want to use recent data which is released using 2020 geographies.

**Strategy to solve:**

1. Read in 2022 ACS data by block group
2. Read in crosswalk weights for 2020 block groups to 2010 tracts 
3. Merge on the weights to the ACS data
4. Weight the population counts (bg_count * bg_weight)
5. Group by 2010 tract and sum the weighted counts
6. Calculate percentages
7. Merge weighted demographics and TIGER GIS data

**Sources:**

- data.census.gov
- https://www.nhgis.org/geographic-crosswalks 
- https://www2.census.gov/geo/tiger/TIGER2010/


In [1]:
import zipfile

import geopandas as gpd
import numpy as np
import pandas as pd

# Constants

In [2]:
RACE_DATA_FP = "./inputs/census_data/ACSDT5Y2022.B03002_2024-02-26T132852.zip"
DP_DATA_FILE_NAME = "ACSDT5Y2022.B03002-Data.csv"
RACE_DATA_COL_MAP = "./inputs/column_mappings/B03002_bg_demographic_data_columns.csv"

CW_DATA_FP = "./inputs/crosswalk/nhgis_bg2020_tr2010_26.zip"
CW_DATA_FILE_NAME = "nhgis_bg2020_tr2010_26.csv"
CW_COL_ID_AND_WEIGHT = ["bg2020ge", "tr2010ge", "wt_pop", "parea"]

TRACTS_2010_GIS_FP = "./inputs/tiger_files/tl_2010_26_tract10.zip"
INDEX_COLS = ["geo_id", "geo_name", "tr2010ge"]
GIS_COL_RENAMES = {"GEOID10": "geo_id", "NAMELSAD10": "geo_name"}

GIS_COL = [
    "geo_id",
    "geo_name",
    "tot_pop_wt",
    "white_tot_wt",
    "black_tot_wt",
    "aian_tot_wt",
    "asian_tot_wt",
    "nhopi_tot_wt",
    "other_tot_wt",
    "two_more_tot_wt",
    "latin_tot_wt",
    "white_pct_wt",
    "black_pct_wt",
    "aian_pct_wt",
    "asian_pct_wt",
    "nhopi_pct_wt",
    "other_pct_wt",
    "two_more_pct_wt",
    "latin_pct_wt",
    "geometry",
]

DATA_EXPORT_FP = "./outputs/tract_demographics_2020_2010.csv"
MAP_EXPORT_FP = "./outputs/tract_demographics_2020_2010.geojson"

In [3]:
def read_data_file(zip_fp, data_file_name, **read_csv_options):
    with zipfile.ZipFile(zip_fp) as z:
        with z.open(data_file_name) as f:
            return pd.read_csv(f, **read_csv_options)

In [4]:
def read_acs_data(
    acs_data_zip_fp, data_file_name, columns_codes_to_extract=None, column_renames=None
):
    df = read_data_file(acs_data_zip_fp, data_file_name, **{"skiprows": [1]})
    if columns_codes_to_extract:
        df = df[columns_codes_to_extract].copy()
    if column_renames:
        df = df.rename(columns=column_renames)
    return df

# Read data

## Demographics

In [5]:
RACE_COL_MAP = pd.read_csv(RACE_DATA_COL_MAP)
RACE_COL_MAP = RACE_COL_MAP.dropna(subset=["RENAME"])

race_col_renames = dict(zip(RACE_COL_MAP["KEY"], RACE_COL_MAP["RENAME"]))

In [6]:
DF_RACE_BG = read_acs_data(
    RACE_DATA_FP, DP_DATA_FILE_NAME, list(RACE_COL_MAP["KEY"]), race_col_renames
)
DF_RACE_BG

,geo_id,geo_name,tot_pop,white_tot,black_tot,aian_tot,asian_tot,nhopi_tot,other_tot,two_more_tot,latin_tot
0,1500000US260010001001,Block Group 1; Census Tract 1; Alcona County; ...,666,591,8,0,0,0,17,34,16
1,1500000US260010001002,Block Group 2; Census Tract 1; Alcona County; ...,1032,988,0,0,0,0,0,24,20
2,1500000US260019701001,Block Group 1; Census Tract 9701; Alcona Count...,997,950,0,0,18,0,5,10,14
3,1500000US260019701002,Block Group 2; Census Tract 9701; Alcona Count...,1409,1284,9,11,2,0,0,52,51
4,1500000US260019704001,Block Group 1; Census Tract 9704; Alcona Count...,1338,1277,2,4,0,0,4,32,19
...,...,...,...,...,...,...,...,...,...,...,...
8381,1500000US261653807002,Block Group 2; Census Tract 3807; Wexford Coun...,1452,1330,29,0,0,0,0,68,25
8382,1500000US261653807003,Block Group 3; Census Tract 3807; Wexford Coun...,449,427,22,0,0,0,0,0,0
8383,1500000US261653808001,Block Group 1; Census Tract 3808; Wexford Coun...,560,524,0,0,0,0,0,11,25
8384,1500000US261653808002,Block Group 2; Census Tract 3808; Wexford Coun...,609,533,13,0,58,0,0,5,0


## Crosswalk

In [7]:
DF_CW_BG_TRACTS = read_data_file(
    CW_DATA_FP, CW_DATA_FILE_NAME, dtype={"bg2020ge": str, "tr2010ge": str}
)
DF_CW_BG_TRACTS

,bg2020gj,bg2020ge,tr2010gj,tr2010ge,wt_pop,wt_adult,wt_hh,wt_hu,parea
0,G26000100001001,260010001001,G2600010000100,26001000100,1.000000,1.00000,1.000000,1.000000,1.000000
1,G26000100001002,260010001002,G2600010970100,26001970100,0.029412,0.02358,0.014981,0.012366,0.017712
2,G26000100001002,260010001002,G2600010000100,26001000100,0.970588,0.97642,0.985019,0.987634,0.982288
3,G26000109701001,260019701001,G2600010970100,26001970100,1.000000,1.00000,1.000000,1.000000,1.000000
4,G26000109701002,260019701002,G2600010970100,26001970100,1.000000,1.00000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
9840,G26016503807002,261653807002,G2601650380700,26165380700,1.000000,1.00000,1.000000,1.000000,1.000000
9841,G26016503807003,261653807003,G2601650380700,26165380700,1.000000,1.00000,1.000000,1.000000,1.000000
9842,G26016503808001,261653808001,G2601650380800,26165380800,1.000000,1.00000,1.000000,1.000000,1.000000
9843,G26016503808002,261653808002,G2601650380800,26165380800,1.000000,1.00000,1.000000,1.000000,1.000000


In [8]:
GDF_TRACTS = gpd.read_file(TRACTS_2010_GIS_FP)
GDF_TRACTS

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,26,111,290500,26111290500,2905,Census Tract 2905,G5020,S,20028764,5451858,+43.5849689,-084.2539377,"POLYGON ((-84.27849 43.60846, -84.27839 43.607..."
1,26,111,290700,26111290700,2907,Census Tract 2907,G5020,S,4890107,50768,+43.6170755,-084.1920233,"POLYGON ((-84.20713 43.61134, -84.20744 43.611..."
2,26,111,290800,26111290800,2908,Census Tract 2908,G5020,S,4472984,3069,+43.6340278,-084.2016894,"POLYGON ((-84.19284 43.64106, -84.18922 43.641..."
3,26,111,290900,26111290900,2909,Census Tract 2909,G5020,S,6522679,4587,+43.6475643,-084.2215974,"POLYGON ((-84.20688 43.65404, -84.20431 43.652..."
4,26,111,291000,26111291000,2910,Census Tract 2910,G5020,S,5704264,5942,+43.6464585,-084.2688725,"POLYGON ((-84.28999 43.65613, -84.28452 43.656..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,26,161,422900,26161422900,4229,Census Tract 4229,G5020,S,2575631,0,+42.1673143,-083.6908217,"POLYGON ((-83.69749 42.16381, -83.69762 42.167..."
2809,26,161,464000,26161464000,4640,Census Tract 4640,G5020,S,88803648,234354,+42.3899612,-083.6066461,"POLYGON ((-83.66238 42.35533, -83.66240 42.356..."
2810,26,161,461000,26161461000,4610,Census Tract 4610,G5020,S,90754816,2079194,+42.3766795,-083.8582595,"POLYGON ((-83.81413 42.34140, -83.81475 42.341..."
2811,26,161,407600,26161407600,4076,Census Tract 4076,G5020,S,2041815,10551,+42.2700183,-083.5938966,"POLYGON ((-83.59079 42.27588, -83.58833 42.275..."


# Merge CW weights

In [9]:
df_cw_weight = DF_CW_BG_TRACTS[CW_COL_ID_AND_WEIGHT].copy()
bg_2020_ge_len = len(df_cw_weight["bg2020ge"][0])
df_cw_weight

,bg2020ge,tr2010ge,wt_pop,parea
0,260010001001,26001000100,1.000000,1.000000
1,260010001002,26001970100,0.029412,0.017712
2,260010001002,26001000100,0.970588,0.982288
3,260019701001,26001970100,1.000000,1.000000
4,260019701002,26001970100,1.000000,1.000000
...,...,...,...,...
9840,261653807002,26165380700,1.000000,1.000000
9841,261653807003,26165380700,1.000000,1.000000
9842,261653808001,26165380800,1.000000,1.000000
9843,261653808002,26165380800,1.000000,1.000000


In [10]:
df_race_bg = DF_RACE_BG.copy()
df_race_bg["geo_id"] = DF_RACE_BG["geo_id"].str.slice(-1 * bg_2020_ge_len)
df_race_bg

,geo_id,geo_name,tot_pop,white_tot,black_tot,aian_tot,asian_tot,nhopi_tot,other_tot,two_more_tot,latin_tot
0,260010001001,Block Group 1; Census Tract 1; Alcona County; ...,666,591,8,0,0,0,17,34,16
1,260010001002,Block Group 2; Census Tract 1; Alcona County; ...,1032,988,0,0,0,0,0,24,20
2,260019701001,Block Group 1; Census Tract 9701; Alcona Count...,997,950,0,0,18,0,5,10,14
3,260019701002,Block Group 2; Census Tract 9701; Alcona Count...,1409,1284,9,11,2,0,0,52,51
4,260019704001,Block Group 1; Census Tract 9704; Alcona Count...,1338,1277,2,4,0,0,4,32,19
...,...,...,...,...,...,...,...,...,...,...,...
8381,261653807002,Block Group 2; Census Tract 3807; Wexford Coun...,1452,1330,29,0,0,0,0,68,25
8382,261653807003,Block Group 3; Census Tract 3807; Wexford Coun...,449,427,22,0,0,0,0,0,0
8383,261653808001,Block Group 1; Census Tract 3808; Wexford Coun...,560,524,0,0,0,0,0,11,25
8384,261653808002,Block Group 2; Census Tract 3808; Wexford Coun...,609,533,13,0,58,0,0,5,0


In [11]:
df_race_bg_cw_merged = df_race_bg.merge(
    df_cw_weight, left_on="geo_id", right_on="bg2020ge", how="outer", validate="1:m"
)
assert df_race_bg_cw_merged["wt_pop"].isnull().sum() == 0
df_race_bg_cw_merged

,geo_id,geo_name,tot_pop,white_tot,black_tot,aian_tot,asian_tot,nhopi_tot,other_tot,two_more_tot,latin_tot,bg2020ge,tr2010ge,wt_pop,parea
0,260010001001,Block Group 1; Census Tract 1; Alcona County; ...,666,591,8,0,0,0,17,34,16,260010001001,26001000100,1.000000,1.000000
1,260010001002,Block Group 2; Census Tract 1; Alcona County; ...,1032,988,0,0,0,0,0,24,20,260010001002,26001970100,0.029412,0.017712
2,260010001002,Block Group 2; Census Tract 1; Alcona County; ...,1032,988,0,0,0,0,0,24,20,260010001002,26001000100,0.970588,0.982288
3,260019701001,Block Group 1; Census Tract 9701; Alcona Count...,997,950,0,0,18,0,5,10,14,260019701001,26001970100,1.000000,1.000000
4,260019701002,Block Group 2; Census Tract 9701; Alcona Count...,1409,1284,9,11,2,0,0,52,51,260019701002,26001970100,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9840,261653807002,Block Group 2; Census Tract 3807; Wexford Coun...,1452,1330,29,0,0,0,0,68,25,261653807002,26165380700,1.000000,1.000000
9841,261653807003,Block Group 3; Census Tract 3807; Wexford Coun...,449,427,22,0,0,0,0,0,0,261653807003,26165380700,1.000000,1.000000
9842,261653808001,Block Group 1; Census Tract 3808; Wexford Coun...,560,524,0,0,0,0,0,11,25,261653808001,26165380800,1.000000,1.000000
9843,261653808002,Block Group 2; Census Tract 3808; Wexford Coun...,609,533,13,0,58,0,0,5,0,261653808002,26165380800,1.000000,1.000000


# Clean data

## Column names

In [12]:
count_col_names = [col for col in df_race_bg_cw_merged.columns if "tot" in col]
count_col_names

['tot_pop',
 'white_tot',
 'black_tot',
 'aian_tot',
 'asian_tot',
 'nhopi_tot',
 'other_tot',
 'two_more_tot',
 'latin_tot']

In [13]:
count_wt_col_names = [col + "_wt" for col in count_col_names]
count_wt_col_names

['tot_pop_wt',
 'white_tot_wt',
 'black_tot_wt',
 'aian_tot_wt',
 'asian_tot_wt',
 'nhopi_tot_wt',
 'other_tot_wt',
 'two_more_tot_wt',
 'latin_tot_wt']

In [14]:
pct_wt_col_names = [col.replace("tot", "pct") for col in count_wt_col_names[1:]]
pct_wt_col_names

['white_pct_wt',
 'black_pct_wt',
 'aian_pct_wt',
 'asian_pct_wt',
 'nhopi_pct_wt',
 'other_pct_wt',
 'two_more_pct_wt',
 'latin_pct_wt']

## Weight total

In [15]:
df_race_bg_cw_merged[count_wt_col_names] = np.round(
    df_race_bg_cw_merged[count_col_names].to_numpy()
    * df_race_bg_cw_merged[["wt_pop"]].to_numpy()
)
df_race_wt = df_race_bg_cw_merged[INDEX_COLS + count_wt_col_names].copy()
df_race_wt

,geo_id,geo_name,tr2010ge,tot_pop_wt,white_tot_wt,black_tot_wt,aian_tot_wt,asian_tot_wt,nhopi_tot_wt,other_tot_wt,two_more_tot_wt,latin_tot_wt
0,260010001001,Block Group 1; Census Tract 1; Alcona County; ...,26001000100,666.0,591.0,8.0,0.0,0.0,0.0,17.0,34.0,16.0
1,260010001002,Block Group 2; Census Tract 1; Alcona County; ...,26001970100,30.0,29.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,260010001002,Block Group 2; Census Tract 1; Alcona County; ...,26001000100,1002.0,959.0,0.0,0.0,0.0,0.0,0.0,23.0,19.0
3,260019701001,Block Group 1; Census Tract 9701; Alcona Count...,26001970100,997.0,950.0,0.0,0.0,18.0,0.0,5.0,10.0,14.0
4,260019701002,Block Group 2; Census Tract 9701; Alcona Count...,26001970100,1409.0,1284.0,9.0,11.0,2.0,0.0,0.0,52.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9840,261653807002,Block Group 2; Census Tract 3807; Wexford Coun...,26165380700,1452.0,1330.0,29.0,0.0,0.0,0.0,0.0,68.0,25.0
9841,261653807003,Block Group 3; Census Tract 3807; Wexford Coun...,26165380700,449.0,427.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0
9842,261653808001,Block Group 1; Census Tract 3808; Wexford Coun...,26165380800,560.0,524.0,0.0,0.0,0.0,0.0,0.0,11.0,25.0
9843,261653808002,Block Group 2; Census Tract 3808; Wexford Coun...,26165380800,609.0,533.0,13.0,0.0,58.0,0.0,0.0,5.0,0.0


## Group by 2010 tract

In [16]:
df_tracts_race_wt = df_race_wt.groupby("tr2010ge").sum(numeric_only=True)
df_tracts_race_wt["tot_pop_wt"] = df_tracts_race_wt[count_wt_col_names[1:]].sum(axis=1)
df_tracts_race_wt

,tot_pop_wt,white_tot_wt,black_tot_wt,aian_tot_wt,asian_tot_wt,nhopi_tot_wt,other_tot_wt,two_more_tot_wt,latin_tot_wt
tr2010ge,,,,,,,,,
26001000100,1667.0,1550.0,8.0,0.0,0.0,0.0,17.0,57.0,35.0
26001970100,2437.0,2263.0,9.0,11.0,20.0,0.0,5.0,63.0,66.0
26001970400,2789.0,2661.0,20.0,17.0,0.0,2.0,4.0,54.0,31.0
26001970500,2057.0,1907.0,3.0,1.0,12.0,0.0,0.0,93.0,41.0
26001970600,1288.0,1222.0,4.0,2.0,0.0,0.0,0.0,44.0,16.0
...,...,...,...,...,...,...,...,...,...
26165380400,4844.0,4486.0,16.0,11.0,24.0,11.0,0.0,226.0,70.0
26165380500,6864.0,6036.0,50.0,38.0,23.0,0.0,0.0,462.0,255.0
26165380600,3275.0,3257.0,14.0,4.0,0.0,0.0,0.0,0.0,0.0


## Calculate percentages

In [17]:
# for tracts with a total population of zero, replace with a placeholder value one to retain data and avoid divide by zero
tot_pop_divisor = df_tracts_race_wt["tot_pop_wt"].replace({0: 1})
df_tracts_race_wt[pct_wt_col_names] = np.round(
    df_tracts_race_wt[count_wt_col_names[1:]].to_numpy()
    / pd.DataFrame(tot_pop_divisor).to_numpy(),
    3,
)
df_tracts_race_wt

,tot_pop_wt,white_tot_wt,black_tot_wt,aian_tot_wt,asian_tot_wt,nhopi_tot_wt,other_tot_wt,two_more_tot_wt,latin_tot_wt,white_pct_wt,black_pct_wt,aian_pct_wt,asian_pct_wt,nhopi_pct_wt,other_pct_wt,two_more_pct_wt,latin_pct_wt
tr2010ge,,,,,,,,,,,,,,,,,
26001000100,1667.0,1550.0,8.0,0.0,0.0,0.0,17.0,57.0,35.0,0.930,0.005,0.000,0.000,0.000,0.010,0.034,0.021
26001970100,2437.0,2263.0,9.0,11.0,20.0,0.0,5.0,63.0,66.0,0.929,0.004,0.005,0.008,0.000,0.002,0.026,0.027
26001970400,2789.0,2661.0,20.0,17.0,0.0,2.0,4.0,54.0,31.0,0.954,0.007,0.006,0.000,0.001,0.001,0.019,0.011
26001970500,2057.0,1907.0,3.0,1.0,12.0,0.0,0.0,93.0,41.0,0.927,0.001,0.000,0.006,0.000,0.000,0.045,0.020
26001970600,1288.0,1222.0,4.0,2.0,0.0,0.0,0.0,44.0,16.0,0.949,0.003,0.002,0.000,0.000,0.000,0.034,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165380400,4844.0,4486.0,16.0,11.0,24.0,11.0,0.0,226.0,70.0,0.926,0.003,0.002,0.005,0.002,0.000,0.047,0.014
26165380500,6864.0,6036.0,50.0,38.0,23.0,0.0,0.0,462.0,255.0,0.879,0.007,0.006,0.003,0.000,0.000,0.067,0.037
26165380600,3275.0,3257.0,14.0,4.0,0.0,0.0,0.0,0.0,0.0,0.995,0.004,0.001,0.000,0.000,0.000,0.000,0.000


# Merge demographics and GIS data

In [18]:
gdf_2020_to_2010_cw = GDF_TRACTS.merge(
    df_tracts_race_wt,
    left_on="GEOID10",
    right_on="tr2010ge",
    validate="1:1",
    how="left",
)
gdf_2020_to_2010_cw = gdf_2020_to_2010_cw.rename(columns=GIS_COL_RENAMES)
gdf_2020_to_2010_cw = gdf_2020_to_2010_cw[GIS_COL].copy()
assert gdf_2020_to_2010_cw["tot_pop_wt"].isnull().sum() == 0
gdf_2020_to_2010_cw

,geo_id,geo_name,tot_pop_wt,white_tot_wt,black_tot_wt,aian_tot_wt,asian_tot_wt,nhopi_tot_wt,other_tot_wt,two_more_tot_wt,latin_tot_wt,white_pct_wt,black_pct_wt,aian_pct_wt,asian_pct_wt,nhopi_pct_wt,other_pct_wt,two_more_pct_wt,latin_pct_wt,geometry
0,26111290500,Census Tract 2905,1904.0,1809.0,0.0,3.0,0.0,0.0,0.0,17.0,75.0,0.950,0.000,0.002,0.000,0.000,0.000,0.009,0.039,"POLYGON ((-84.27849 43.60846, -84.27839 43.607..."
1,26111290700,Census Tract 2907,3961.0,3267.0,52.0,15.0,369.0,0.0,0.0,57.0,201.0,0.825,0.013,0.004,0.093,0.000,0.000,0.014,0.051,"POLYGON ((-84.20713 43.61134, -84.20744 43.611..."
2,26111290800,Census Tract 2908,5139.0,4635.0,88.0,0.0,23.0,0.0,0.0,135.0,258.0,0.902,0.017,0.000,0.004,0.000,0.000,0.026,0.050,"POLYGON ((-84.19284 43.64106, -84.18922 43.641..."
3,26111290900,Census Tract 2909,7927.0,6837.0,251.0,0.0,578.0,0.0,0.0,120.0,141.0,0.862,0.032,0.000,0.073,0.000,0.000,0.015,0.018,"POLYGON ((-84.20688 43.65404, -84.20431 43.652..."
4,26111291000,Census Tract 2910,6421.0,5386.0,83.0,0.0,52.0,0.0,13.0,226.0,661.0,0.839,0.013,0.000,0.008,0.000,0.002,0.035,0.103,"POLYGON ((-84.28999 43.65613, -84.28452 43.656..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,26161422900,Census Tract 4229,1845.0,1010.0,573.0,34.0,2.0,2.0,9.0,96.0,119.0,0.547,0.311,0.018,0.001,0.001,0.005,0.052,0.064,"POLYGON ((-83.69749 42.16381, -83.69762 42.167..."
2809,26161464000,Census Tract 4640,6973.0,6050.0,161.0,13.0,337.0,0.0,12.0,332.0,68.0,0.868,0.023,0.002,0.048,0.000,0.002,0.048,0.010,"POLYGON ((-83.66238 42.35533, -83.66240 42.356..."
2810,26161461000,Census Tract 4610,7408.0,6569.0,0.0,16.0,190.0,0.0,7.0,381.0,245.0,0.887,0.000,0.002,0.026,0.000,0.001,0.051,0.033,"POLYGON ((-83.81413 42.34140, -83.81475 42.341..."
2811,26161407600,Census Tract 4076,3195.0,1683.0,1120.0,0.0,93.0,22.0,17.0,86.0,174.0,0.527,0.351,0.000,0.029,0.007,0.005,0.027,0.054,"POLYGON ((-83.59079 42.27588, -83.58833 42.275..."


# Export

In [19]:
df_tracts_race_wt.to_csv(DATA_EXPORT_FP)
df_tracts_race_wt

,tot_pop_wt,white_tot_wt,black_tot_wt,aian_tot_wt,asian_tot_wt,nhopi_tot_wt,other_tot_wt,two_more_tot_wt,latin_tot_wt,white_pct_wt,black_pct_wt,aian_pct_wt,asian_pct_wt,nhopi_pct_wt,other_pct_wt,two_more_pct_wt,latin_pct_wt
tr2010ge,,,,,,,,,,,,,,,,,
26001000100,1667.0,1550.0,8.0,0.0,0.0,0.0,17.0,57.0,35.0,0.930,0.005,0.000,0.000,0.000,0.010,0.034,0.021
26001970100,2437.0,2263.0,9.0,11.0,20.0,0.0,5.0,63.0,66.0,0.929,0.004,0.005,0.008,0.000,0.002,0.026,0.027
26001970400,2789.0,2661.0,20.0,17.0,0.0,2.0,4.0,54.0,31.0,0.954,0.007,0.006,0.000,0.001,0.001,0.019,0.011
26001970500,2057.0,1907.0,3.0,1.0,12.0,0.0,0.0,93.0,41.0,0.927,0.001,0.000,0.006,0.000,0.000,0.045,0.020
26001970600,1288.0,1222.0,4.0,2.0,0.0,0.0,0.0,44.0,16.0,0.949,0.003,0.002,0.000,0.000,0.000,0.034,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165380400,4844.0,4486.0,16.0,11.0,24.0,11.0,0.0,226.0,70.0,0.926,0.003,0.002,0.005,0.002,0.000,0.047,0.014
26165380500,6864.0,6036.0,50.0,38.0,23.0,0.0,0.0,462.0,255.0,0.879,0.007,0.006,0.003,0.000,0.000,0.067,0.037
26165380600,3275.0,3257.0,14.0,4.0,0.0,0.0,0.0,0.0,0.0,0.995,0.004,0.001,0.000,0.000,0.000,0.000,0.000


In [20]:
gdf_2020_to_2010_cw.to_file(MAP_EXPORT_FP, index=False, driver="GeoJSON")
gdf_2020_to_2010_cw

,geo_id,geo_name,tot_pop_wt,white_tot_wt,black_tot_wt,aian_tot_wt,asian_tot_wt,nhopi_tot_wt,other_tot_wt,two_more_tot_wt,latin_tot_wt,white_pct_wt,black_pct_wt,aian_pct_wt,asian_pct_wt,nhopi_pct_wt,other_pct_wt,two_more_pct_wt,latin_pct_wt,geometry
0,26111290500,Census Tract 2905,1904.0,1809.0,0.0,3.0,0.0,0.0,0.0,17.0,75.0,0.950,0.000,0.002,0.000,0.000,0.000,0.009,0.039,"POLYGON ((-84.27849 43.60846, -84.27839 43.607..."
1,26111290700,Census Tract 2907,3961.0,3267.0,52.0,15.0,369.0,0.0,0.0,57.0,201.0,0.825,0.013,0.004,0.093,0.000,0.000,0.014,0.051,"POLYGON ((-84.20713 43.61134, -84.20744 43.611..."
2,26111290800,Census Tract 2908,5139.0,4635.0,88.0,0.0,23.0,0.0,0.0,135.0,258.0,0.902,0.017,0.000,0.004,0.000,0.000,0.026,0.050,"POLYGON ((-84.19284 43.64106, -84.18922 43.641..."
3,26111290900,Census Tract 2909,7927.0,6837.0,251.0,0.0,578.0,0.0,0.0,120.0,141.0,0.862,0.032,0.000,0.073,0.000,0.000,0.015,0.018,"POLYGON ((-84.20688 43.65404, -84.20431 43.652..."
4,26111291000,Census Tract 2910,6421.0,5386.0,83.0,0.0,52.0,0.0,13.0,226.0,661.0,0.839,0.013,0.000,0.008,0.000,0.002,0.035,0.103,"POLYGON ((-84.28999 43.65613, -84.28452 43.656..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,26161422900,Census Tract 4229,1845.0,1010.0,573.0,34.0,2.0,2.0,9.0,96.0,119.0,0.547,0.311,0.018,0.001,0.001,0.005,0.052,0.064,"POLYGON ((-83.69749 42.16381, -83.69762 42.167..."
2809,26161464000,Census Tract 4640,6973.0,6050.0,161.0,13.0,337.0,0.0,12.0,332.0,68.0,0.868,0.023,0.002,0.048,0.000,0.002,0.048,0.010,"POLYGON ((-83.66238 42.35533, -83.66240 42.356..."
2810,26161461000,Census Tract 4610,7408.0,6569.0,0.0,16.0,190.0,0.0,7.0,381.0,245.0,0.887,0.000,0.002,0.026,0.000,0.001,0.051,0.033,"POLYGON ((-83.81413 42.34140, -83.81475 42.341..."
2811,26161407600,Census Tract 4076,3195.0,1683.0,1120.0,0.0,93.0,22.0,17.0,86.0,174.0,0.527,0.351,0.000,0.029,0.007,0.005,0.027,0.054,"POLYGON ((-83.59079 42.27588, -83.58833 42.275..."
